In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    google_api_key=os.environ["GOOGLE_API_KEY"]
)



In [2]:
response = llm.invoke("Write a few lines on Generative AI.")
print(response.content)

Generative AI is a type of artificial intelligence that can create new content, ranging from text and images to music and code.  It learns patterns from existing data and then uses this knowledge to generate similar, yet novel outputs.  This technology has the potential to revolutionize creative industries, automate tasks, and personalize user experiences, though it also raises ethical concerns around misuse and authenticity.


In [3]:
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase

In [4]:
db_user = "root"
db_password = "Charan5507"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [44]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
response

'```sql\nSELECT COUNT(DISTINCT `CustomerID`)\nFROM `sales_tb`;\n```'

In [45]:
cleaned_query = response.strip('```sql\n').strip('\n```')
print(cleaned_query)

SELECT COUNT(DISTINCT `CustomerID`)
FROM `sales_tb`;


In [47]:
result = db.run(cleaned_query)
print(result)

[(29,)]


In [48]:
chain = create_sql_query_chain(llm, db)
def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("###################################################")        
        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")


In [49]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

```sql
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomerCount FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
```
###################################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS UniqueCustomerCount FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [50]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)


```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount` FROM `sales_tb` GROUP BY `ProductCategory`;
```

```
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount` FROM `sales_tb` GROUP BY `ProductCategory`;
###################################################
###################################################
[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]


In [51]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)


```sql
SELECT `Gender`, AVG(`Age`) AS AverageAge
FROM `sales_tb`
GROUP BY `Gender`;
```
###################################################
SELECT `Gender`, AVG(`Age`) AS AverageAge
FROM `sales_tb`
GROUP BY `Gender`;
###################################################
###################################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [52]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

Question: identify the top spending customers based on their total amount spent.
SQLQuery: ```sql
SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent`
FROM `sales_tb`
GROUP BY `CustomerID`
ORDER BY `TotalSpent` DESC
LIMIT 5;
```
###################################################
Question: identify the top spending customers based on their total amount spent.
SQLQuery: ```sql
SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent`
FROM `sales_tb`
GROUP BY `CustomerID`
ORDER BY `TotalSpent` DESC
LIMIT 5;
###################################################


NameError: name 'ProgrammingError' is not defined

In [53]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

```sql
SELECT MONTH(`Date`) AS Month, COUNT(*) AS TransactionCount FROM `sales_tb` GROUP BY Month ORDER BY TransactionCount DESC LIMIT 5;
```
###################################################
SELECT MONTH(`Date`) AS Month, COUNT(*) AS TransactionCount FROM `sales_tb` GROUP BY Month ORDER BY TransactionCount DESC LIMIT 5;
###################################################
###################################################
[(4, 5), (2, 4), (1, 4), (10, 4), (11, 3)]


In [54]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)


```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) AS TotalSalesAmount, AVG(`PriceperUnit`) AS AveragePricePerUnit FROM `sales_tb` GROUP BY `ProductCategory`;
```
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS TotalSalesAmount, AVG(`PriceperUnit`) AS AveragePricePerUnit FROM `sales_tb` GROUP BY `ProductCategory`;
###################################################
###################################################
[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]
